In [346]:
import numpy as np
import re
import heapq
from typing import List

In [347]:
f = open("./input", "r")
lines = f.read().strip().splitlines()

In [348]:
from operator import add, sub
def tsub(a, b):
    return tuple(map(sub, a, b))
def tadd(a, b):
    return tuple(map(add, a, b))
def print_grid(g):
    print("\n".join(["".join(x) for x in g]), flush=True)

In [350]:
NEIGHBOURS = [(0, 1), (0, -1), (1, 0), (-1, 0)]
MAZE = np.array([[x for x in l] for l in lines])
START = tuple(zip(*np.where(MAZE == "S")))[0]
END = tuple(zip(*np.where(MAZE == "E")))[0]

class GridNode:
    def __init__(self, position: tuple, direction: tuple, parent=None):
        self.direction = direction
        self._parent = None
        self.value = None
        self.position = position
        self.parent = parent

    @property
    def parent(self):
        return self._parent

    @parent.setter
    def parent(self, parent: "GridNode"):
        self._parent = parent
        if not parent:
            self.value = 0
            return
        self.value = parent.value + (1 if parent.direction == self.direction else 1001)

    def __eq__(self, other: "GridNode"):
        if not isinstance(other, GridNode):
            raise TypeError("not GridNode")
        result = self.position == other.position
        return result

    def __add__(self, other: "GridNode"):
        if isinstance(other, GridNode):
            return self.position + other.position
        return self.position + other

    def __lt__(self, other: "GridNode"):
        return self.value < other.value

    def __gt__(self, other: "GridNode"):
        return self.value > other.value

    def __str__(self):
        return str(self.position) + " " + str(self.direction)

    def __repr__(self):
        return self.__str__()

    def get_directions(self):
        if self.direction is None:
            raise ValueError("node not closed")
        d = []
        for n in NEIGHBOURS:
            if MAZE[tadd(n, self.position)] == "#":
                continue
            d.append(n)
        return d

In [358]:
start_node = GridNode(START, (0, 1))
open_list: List[GridNode] = []
open_list.append(start_node)
closed_list: List[GridNode] = []
final_path: List[GridNode] = []

while len(open_list) > 0:
    current_node: GridNode = heapq.heappop(open_list)
    closed_list.append(current_node)

    if current_node.position == END:
        current = current_node
        while current is not None:
            final_path.append(current)
            current = current.parent
        final_path.reverse()
        break

    for direction in current_node.get_directions():
        new_position = tadd(current_node.position, direction)
        new_node = GridNode(new_position, direction, current_node)

        if new_node in closed_list or new_node in open_list:
            continue

        heapq.heappush(open_list, new_node)

print(final_path[-1].value)

65436


In [ ]:
alternate_path = []
from tqdm import tqdm
for start_node in tqdm(final_path):
    open_list: List[GridNode] = []
    for dir in start_node.get_directions():
        new = GridNode(tadd(dir, start_node.position), dir, start_node)
        if new not in final_path:
            open_list.append(new)

    closed_list: List[GridNode] = []

    while len(open_list) > 0:
        current_node: GridNode = heapq.heappop(open_list)
        closed_list.append(current_node)

        if current_node in final_path:
            final_path_node_index = final_path.index(current_node)
            final_path_node = final_path[final_path_node_index]
            # check if n+1 will do turn, if yes add 1000 to value
            adjust = (
                1000
                if len(final_path) > final_path_node_index + 1
                and final_path[final_path_node_index + 1].direction is current_node.direction
                and final_path_node.direction is not current_node.direction
                else 0
            )
            if current_node.value == final_path_node.value + adjust:
                current = current_node
                new_alt_path: List[GridNode] = []
                while True:
                    if current == start_node:
                        break
                    new_alt_path.append(current)
                    current = current.parent
                new_alt_path.reverse()
                alternate_path.append(new_alt_path)
            continue

        for direction in current_node.get_directions():
            new_position = tadd(current_node.position, direction)
            new_node = GridNode(new_position, direction, current_node)
            if new_node in closed_list or new_node in open_list:
                continue

            heapq.heappush(open_list, new_node)

100%|██████████| 437/437 [08:19<00:00,  1.14s/it]


In [355]:
best_path_tiles = set([x.position for x in final_path])
for a in alternate_path:
    for p in a:
        best_path_tiles.add(p.position)

len(best_path_tiles)

489